<a href="https://colab.research.google.com/github/harsh-hy/DeepLearning/blob/main/CIFAR100_CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
(x_train,_),(_,_)=tf.keras.datasets.cifar100.load_data()

In [ ]:
x_train=(x_train.astype("float32")-127.5)/127.5

In [ ]:
BUFFER_SIZE=50000
BATCH_SIZE=128
LATENT_DIM=100
dataset=tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def make_generator():
  model=tf.keras.Sequential([
      layers.Dense(8*8*256,input_shape=(LATENT_DIM,)),
      layers.LeakyReLU(0.2),
      layers.Reshape((8,8,256)),

      layers.Conv2DTranspose(128,4,strides=2,padding="same"),
      layers.LeakyReLU(0.2),

      layers.Conv2DTranspose(64,4,strides=2,padding="same"),
      layers.LeakyReLU(0.2),

      layers.Conv2D(3, 3, padding="same", activation="tanh")
  ])
  return model

In [ ]:
def make_discriminator():
  model=tf.keras.Sequential([
      layers.Conv2D(64,4,strides=2,padding="same",input_shape=(32,32,3)),
      layers.LeakyReLU(0.2),
      layers.Dropout(0.3),

      layers.Conv2D(128,4,strides=2,padding="same"),
      layers.LeakyReLU(0.2),
      layers.Dropout(0.3),

      layers.Flatten(),
      layers.Dense(1, activation="sigmoid")
  ])
  return model

In [ ]:
generator=make_generator()
discriminator=make_discriminator()

In [ ]:
discriminator.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
discriminator.trainable=False

In [ ]:
gan_input=layers.Input(shape=(LATENT_DIM,))
gan_output=discriminator(generator(gan_input))
gan=tf.keras.Model(gan_input,gan_output)
gan.compile(loss="binary_crossentropy",optimizer="adam")

In [ ]:
EPOCHS=2000
SAMPLE_INTERVAL=500
half_batch=BATCH_SIZE//2

In [ ]:
for epoch in range(1,EPOCHS+1):
  noise=np.random.normal(0,1,(half_batch, LATENT_DIM))
  gen_imgs=generator.predict(noise)

  idx=np.random.randint(0,x_train.shape[0],half_batch)
  real_imgs=x_train[idx]

  d_loss_real=discriminator.train_on_batch(real_imgs,np.ones((half_batch,1)))
  d_loss_fake=discriminator.train_on_batch(gen_imgs,np.zeros((half_batch,1)))
  d_loss=0.5*np.add(d_loss_real,d_loss_fake)

  noise=np.random.normal(0,1,(BATCH_SIZE, LATENT_DIM))
  g_loss=gan.train_on_batch(noise,np.ones((BATCH_SIZE,1)))

  if epoch%100==0:
    print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

  if epoch%SAMPLE_INTERVAL==0:
    noise=np.random.normal(0,1,(16, LATENT_DIM))
    gen_imgs=generator.predict(noise)
    gen_imgs=0.5*gen_imgs+0.5

    plt.figure(figsize=(10,10))
    for i in range(9):
      plt.subplot(3,3,i+1)
      plt.imshow(gen_imgs[i])
      plt.axis("off")
    plt.subtitle(f"Epoch {epoch}")
    plt.show()